In [1]:
import pandas as pd
import re
import pymorphy2
import math
morph = pymorphy2.MorphAnalyzer()

In [19]:
from wiki_ru_wordnet import WikiWordnet
wikiwordnet = WikiWordnet()

In [ ]:
def get_simular_worsd(word):
    lemmas = []
    synsets = wikiwordnet.get_synsets(word)
    for synset in synsets:
        for w in synset.get_words():
            lemmas.append(w.lemma())
            for hyponym in wikiwordnet.get_hyponyms(synset):
                lemmas.append(hyponym.get_words()[0].lemma())
            

In [10]:
df = pd.read_excel('Датасет. Задача 1.xlsx')
df

,Проект,Номинация / Задача,Название,категория,Иновация
0,Новатор Москвы,Лидеры инноваций,Экологический мониторинг атмосферного воздуха,IT,+
1,Новатор Москвы,Лидеры инноваций,Облачный сервис мониторинга и управления борто...,IT,+
2,Новатор Москвы,Лидеры инноваций,Платформа тестирования и повышения уровня цифр...,IT,+
3,Новатор Москвы,Меняющие реальность,Экологически безопасная и энергоэффективная те...,Технологии и производство,+
4,Новатор Москвы,Меняющие реальность,Перовскитные солнечные батареи для питания пор...,Технологии и производство,+
5,Новатор Москвы,Меняющие реальность,"Платформа для анализа логов, UX-мониторинга и ...",IT,+
6,Новатор Москвы,Меняющие реальность,Интерактивные прозрачные дисплеи,Технологии и производство,+
7,Новатор Москвы,Меняющие реальность,Высокоэффективные композиционные материалы на ...,Новые материалы,+
8,Новатор Москвы,Проект будущего,Разработка биосовместимого полимер- композицио...,Новые материалы,+
9,Новатор Москвы,Проект будущего,Новые люминесцентные материалы для биологическ...,Новые материалы,+


In [29]:
def text_to_words(text):
    text = re.sub("[,|-|(|)|.|;|:|1|2|3|4|5|6|7|8|9|0]","", text)
    text = text.replace('-', ' ')
    text = text.lower()
    words = text.split(' ')
    words = map(morph.normal_forms, words)
    words = list(map(lambda x: x[0], words))
    words = list(set(words))
    return words
df['words'] = df['Название'].apply(text_to_words)

In [30]:
df_woc = pd.DataFrame({'Inovation': pd.DataFrame(df['Иновация'].value_counts()).index})
df_woc

,Inovation
0,+
1,-


In [31]:
# create clouds for each category
def create_clouds(category):
    df_c = df[df['Иновация'] == category]
    claud = []
    df_c['words'].apply(lambda x: claud.extend(x))
    return claud

df_woc['clouds'] = df_woc['Inovation'].apply(create_clouds)

In [32]:
def clear_clouds(row):
    category, cloud = row['Inovation'], row['clouds']
    new_cloud = cloud.copy()
    for w in cloud:
        other_clouds = list(df_woc[df_woc['Inovation'] != category]['clouds'])
        for orher_cloud in other_clouds:
            if w in orher_cloud:
                new_cloud.pop(new_cloud.index(w))
                break
    return new_cloud

df_woc['clear_clouds'] = df_woc.apply(clear_clouds, axis=1)

In [33]:
df_woc

,Inovation,clouds,clear_clouds
0,+,"[мониторинг, экологический, атмосферный, возду...","[мониторинг, экологический, атмосферный, возду..."
1,-,"[она, постановка, предварительный, ознакомитьс...","[постановка, предварительный, ознакомиться, де..."


In [34]:
df_test = pd.read_excel('test_dataset.xlsx')
df_test

,Название
0,Платформа для содействия внедрению новых техно...
1,Производство иновационных материалов на основе...
2,Сайт для повышения цыфровой грамотности среи п...
3,Система распознаваиния преступления на основе ...
4,Платформа по контролю озеленения
5,Интернет магазин пр продаже игрушек


In [35]:
K_cloud = 1
K_clear_cloud = 10


def find_categoris(text):
    scores = {} # catedory: score
    words = text_to_words(text)
    categoris = list(pd.DataFrame(df_woc['Inovation'].value_counts()).index)
    for c in categoris:
        score = 0
        cloud = list(df_woc[df_woc['Inovation'] == c]['clouds'])[0]
        clear_cloud = list(df_woc[df_woc['Inovation'] == c]['clear_clouds'])[0]
        for w in words:
            if w in cloud:
                score += K_cloud / math.log(len(cloud))
            if w in clear_cloud:
                score += K_clear_cloud / math.log(len(clear_cloud))
        scores[c] = score
    sorted_scores = dict(sorted(scores.items(),key=lambda item:item[1])[::-1])
    return sorted_scores

df_test['scores'] = df_test['Название'].apply(find_categoris)

In [36]:
df_test

,Название,scores
0,Платформа для содействия внедрению новых техно...,"{'+': 8.812795049160343, '-': 0}"
1,Производство иновационных материалов на основе...,"{'+': 3.681647782238219, '-': 0.24136313830927..."
2,Сайт для повышения цыфровой грамотности среи п...,"{'+': 7.050236039328275, '-': 0}"
3,Система распознаваиния преступления на основе ...,"{'+': 7.363295564476439, '-': 0.48272627661855..."
4,Платформа по контролю озеленения,"{'+': 1.9190887724061503, '-': 0.2413631383092..."
5,Интернет магазин пр продаже игрушек,"{'-': 6.444082764981312, '+': 0.15652976257408..."
